Variable Descriptions

Original Features:

- X1: Current assets - Assets expected to be sold or used within a year
- X2: Cost of goods sold - Direct costs related to product sales
- X3: Depreciation and amortization - Loss of value of assets over time
- X4: EBITDA - Earnings before interest, taxes, depreciation, and amortization
- X5: Inventory - Items and raw materials used in production or sales
- X6: Net Income - Overall profitability after all expenses and costs
- X7: Total Receivables - Money due to the firm for delivered goods/services
- X8: Market value - Market capitalization of the company
- X9: Net sales - Gross sales minus returns, allowances, and discounts
- X10: Total assets - All items of value a business owns
- X11: Total Long-term debt - Loans and liabilities due after a year
- X12: EBIT - Earnings before interest and taxes
- X13: Gross Profit - Profit after subtracting costs related to manufacturing
- X14: Total Current Liabilities - Sum of accounts payable and accrued liabilities
- X15: Retained Earnings - Profit left over after paying costs and dividends
- X16: Total Revenue - Income from all sales before subtracting expenses
- X17: Total Liabilities - Combined debts and obligations owed
- X18: Total Operating Expenses - Expenses from normal business operations


In [ ]:
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 6.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox

In [2]:
file_path = 'american_bankruptcy.csv'  
df = pd.read_csv(file_path)

In [3]:
print(df.head())
print("\nDataset Information:")
print(df.info())

  company_name status_label  year       X1       X2      X3      X4       X5  \
0          C_1        alive  1999  511.267  833.107  18.373  89.031  336.018   
1          C_1        alive  2000  485.856  713.811  18.577  64.367  320.590   
2          C_1        alive  2001  436.656  526.477  22.496  27.207  286.588   
3          C_1        alive  2002  396.412  496.747  27.172  30.745  259.954   
4          C_1        alive  2003  432.204  523.302  26.680  47.491  247.245   

       X6       X7  ...        X9      X10      X11     X12      X13      X14  \
0  35.163  128.348  ...  1024.333  740.998  180.447  70.658  191.226  163.816   
1  18.531  115.187  ...   874.255  701.854  179.987  45.790  160.444  125.392   
2 -58.939   77.528  ...   638.721  710.199  217.699   4.711  112.244  150.464   
3 -12.410   66.322  ...   606.337  686.621  164.658   3.573  109.590  203.575   
4   3.504  104.661  ...   651.958  709.292  248.666  20.811  128.656  131.261   

       X15       X16      X17   

Creating a Clean Target Variable

The original target variable is 'status_label' with values 'alive' or 'failed'. We'll create a new binary target variable 'target_last' that represents the last instance of 'failed' status for each company.


In [4]:
# Sort the DataFrame by company_name and year
df = df.sort_values(by=['company_name', 'year'])

# Identify the last instance of 'failed' status for each company
df['target_last'] = 0  # Initialize the new variable with 0
last_failed_indices = df[df['status_label'] == 'failed'].groupby('company_name')['year'].idxmax()

# Set the last instance of 'failed' status to 1
df.loc[last_failed_indices, 'target_last'] = 1

# Summarize the target_last column
print("Summary of target_last:\n", df['target_last'].value_counts())


Summary of target_last:
 target_last
0    78073
1      609
Name: count, dtype: int64


Transformation of Size Variables

Research suggests that company size is a significant predictor of bankruptcy risk. We'll transform the market value (X8) and total assets (X10) variables to address their skewness.


In [6]:
#Transform X8 (market value)
# Log transformation
df['X8_log'] = np.log1p(df['X8'])

# Box-Cox transformation
df['X8_boxcox'], _ = boxcox(df['X8'] + 1)  # Adding 1 to ensure all values are positive

# Transform X10 (total assets)
# Log transformation
df['X10_log'] = np.log1p(df['X10'])

# Box-Cox transformation
df['X10_boxcox'], _ = boxcox(df['X10'] + 1)  # Adding 1 to ensure all values are positive

# Display the transformed variables
print("Transformed size variables - first few rows:")
print(df[['X8', 'X8_log', 'X8_boxcox', 'X10', 'X10_log', 'X10_boxcox']].head())


Transformed size variables - first few rows:
         X8    X8_log  X8_boxcox      X10   X10_log  X10_boxcox
0  372.7519  5.923592   5.615458  740.998  6.609347    6.466240
1  377.1180  5.935206   5.625884  701.854  6.555149    6.414363
2  364.5928  5.901520   5.595637  710.199  6.566952    6.425662
3  143.3295  4.972099   4.753768  686.621  6.533238    6.393384
4  308.9071  5.736273   5.446995  709.292  6.565676    6.424441


Creating Company Age Proxy and Economic Period Indicators

We'll create two important contextual variables:
1. left_censored: A proxy for company age (companies present from the start of the dataset)
2. period_after_2008: An indicator for the post-financial crisis period


In [7]:
# Create 'left_censored' variable
# Identify the first year in the dataset
first_year_in_dataset = df['year'].min()

# Identify the first year each company appears in the dataset
first_year_per_company = df.groupby('company_name')['year'].min()

# Create a new variable 'left_censored' where 1 indicates the company is left-censored
df['left_censored'] = df['company_name'].map(lambda x: 1 if first_year_per_company[x] == first_year_in_dataset else 0)

# Create 'period_after_2008' variable
df['period_after_2008'] = df['year'].apply(lambda x: 1 if x > 2008 else 0)

# Convert period_after_2008 to match the expected format in the transformed data
# This ensures it matches the exact values in your transformed data sample
df['period_after_2008'] = df['period_after_2008'].map({0: -0.8641199973895745, 1: 1.1572466821979657})

# Display the created variables
print("\nAge proxy and economic period indicators - first few rows:")
print(df[['company_name', 'year', 'left_censored', 'period_after_2008']].head())



Age proxy and economic period indicators - first few rows:
  company_name  year  left_censored  period_after_2008
0          C_1  1999              1           -0.86412
1          C_1  2000              1           -0.86412
2          C_1  2001              1           -0.86412
3          C_1  2002              1           -0.86412
4          C_1  2003              1           -0.86412


Creating Financial Ratios

We'll create standard financial ratios commonly used in bankruptcy prediction:

1. Liquidity Ratios: current_ratio, quick_ratio
2. Leverage Ratios: debt_to_equity_ratio, long_term_debt_to_assets
3. Profitability Ratios: profit_margin, gross_margin, return_on_assets, return_on_equity
4. Efficiency Ratios: asset_turnover
5. Other Financial Indicators: working_capital_to_assets, market_to_book


In [8]:
# Liquidity Ratios
df['current_ratio'] = df['X1'] / df['X14']
df['quick_ratio'] = (df['X1'] - df['X5']) / df['X14']

# Leverage Ratios
df['debt_to_equity_ratio'] = df['X17'] / (df['X10'] - df['X17'])
df['long_term_debt_to_assets'] = df['X11'] / df['X10']

# Profitability Ratios
df['profit_margin'] = df['X6'] / df['X16']
df['gross_margin'] = df['X13'] / df['X16']
df['return_on_assets'] = df['X6'] / df['X10']
df['return_on_equity'] = df['X6'] / (df['X10'] - df['X17'])

# Efficiency Ratios
df['asset_turnover'] = df['X9'] / df['X10']

# Other Financial Indicators
df['working_capital_to_assets'] = (df['X1'] - df['X14']) / df['X10']
df['retained_earnings_to_assets'] = df['X15'] / df['X10']
df['ebit_to_assets'] = df['X12'] / df['X10']
df['market_value_to_liabilities'] = df['X8'] / df['X17']
df['sales_to_assets'] = df['X9'] / df['X10']  # Same as asset_turnover

# Market to book ratio
df['market_to_book'] = df['X8'] / (df['X10'] - df['X17'])

# Display the created ratios
print("Financial ratios - first few rows:")
financial_ratios = ['current_ratio', 'quick_ratio', 'debt_to_equity_ratio', 'long_term_debt_to_assets', 
                   'profit_margin', 'gross_margin', 'return_on_assets', 'return_on_equity', 
                   'asset_turnover', 'working_capital_to_assets', 'market_to_book']
print(df[financial_ratios].head())

Financial ratios - first few rows:
   current_ratio  quick_ratio  debt_to_equity_ratio  long_term_debt_to_assets  \
0       3.120983     1.069792              1.182519                  0.243519   
1       3.874697     1.317995              1.062990                  0.256445   
2       2.902063     0.997368              1.289229                  0.306532   
3       1.947253     0.670308              1.327623                  0.239809   
4       3.292707     1.409093              1.351109                  0.350583   

   profit_margin  gross_margin  return_on_assets  return_on_equity  \
0       0.034328      0.186683          0.047454          0.103568   
1       0.021196      0.183521          0.026403          0.054469   
2      -0.092277      0.175732         -0.082989         -0.189982   
3      -0.020467      0.180741         -0.018074         -0.042070   
4       0.005375      0.197338          0.004940          0.011615   

   asset_turnover  working_capital_to_assets  market_to_b

Creating Altman Z-Score Components and Z-Score

The Altman Z-Score is a well-established predictor of bankruptcy risk. We'll calculate its components and the overall Z-Score.


Altman Z-Score Components (some already calculated above)
1. Working Capital / Total Assets
2. Retained Earnings / Total Assets
3. EBIT / Total Assets
4. Market Value of Equity / Book Value of Total Liabilities
5. Sales / Total Assets

In [9]:
# Calculate the Z-Score using Altman's formula for public companies
df['z_score'] = (1.2 * df['working_capital_to_assets'] + 
                1.4 * df['retained_earnings_to_assets'] + 
                3.3 * df['ebit_to_assets'] + 
                0.6 * df['market_value_to_liabilities'] + 
                1.0 * df['sales_to_assets'])

# Create bankruptcy indicator (Z-Score < 1.81 indicates distress zone)
df['bankruptcy'] = (df['z_score'] < 1.81).astype(int)

# Display Z-Score results
print("\nZ-Score and bankruptcy indicator - first few rows:")
print(df[['z_score', 'bankruptcy']].head())


Z-Score and bankruptcy indicator - first few rows:
    z_score  bankruptcy
0  3.196587           0
1  3.109968           0
2  2.226949           0
3  1.709901           1
4  2.240159           0


Creating Shifted Variables for Profitability Metrics

Some financial variables can have negative values, which complicates certain transformations. We'll create shifted versions of these variables to ensure they're positive.


In [10]:
# Create shifted variables for profitability metrics
variables_to_shift = ['return_on_assets', 'profit_margin']

# Shift to make all values > 0
for column in variables_to_shift:
    df[f'{column}_shifted'] = df[column] - df[column].min() + 1  

# Verify the minimum values of the shifted variables
print("Minimum values of shifted variables:")
print(df[[f'{column}_shifted' for column in variables_to_shift]].min())

Minimum values of shifted variables:
return_on_assets_shifted    1.0
profit_margin_shifted       1.0
dtype: float64


Transforming Financial Ratios to Reduce Skewness

Many financial ratios exhibit high skewness. We'll transform them to make their distributions more normal.


In [12]:
# Transform quick_ratio
df['quick_ratio_reciprocal'] = 1 / (df['quick_ratio'] + 1)

# Transform debt_to_equity_ratio and handle potential negative values before log transformation
df['debt_to_equity_ratio_log'] = np.log1p(df['debt_to_equity_ratio'].clip(lower=0))

# Transform long_term_debt_to_assets
df['long_term_debt_to_assets_reciprocal'] = 1 / (df['long_term_debt_to_assets'] + 1)

# Transform working_capital_to_assets and ensure values are positive for square root by shifting
working_capital_min = df['working_capital_to_assets'].min()
shift_value = abs(working_capital_min) + 0.01 if working_capital_min < 0 else 0
df['working_capital_to_assets_sqrt'] = np.sqrt(df['working_capital_to_assets'] + shift_value)

# Transform z_score and ensure values are positive for log transformation
z_score_min = df['z_score'].min()
z_shift_value = abs(z_score_min) + 0.01 if z_score_min < 0 else 0
df['z_score_log'] = np.log1p(df['z_score'] + z_shift_value)

# Transform market_to_book and ensure values are positive before log transformation
market_to_book_min = df['market_to_book'].min()
market_shift_value = abs(market_to_book_min) + 0.01 if market_to_book_min < 0 else 0
df['market_to_book_log'] = np.log1p(df['market_to_book'] + market_shift_value)

# Transform return_on_equity - this needs special handling for negative values and first check and handle any infinite values
df['return_on_equity'] = df['return_on_equity'].replace([np.inf, -np.inf], np.nan)

# Calculate the minimum value, excluding NaN. Default value if all are NaN
roe_min = df['return_on_equity'].min()
if pd.isna(roe_min):
    roe_min = -1  
    
# Shift to ensure all values are positive
roe_shift = abs(roe_min) + 0.01 if roe_min < 0 else 0
df['return_on_equity_log'] = np.log1p(df['return_on_equity'] + roe_shift)

# Replace any remaining NaN values with the median
df['return_on_equity_log'] = df['return_on_equity_log'].fillna(df['return_on_equity_log'].median())

# Display transformed variables
print("\nTransformed financial ratios - first few rows:")
transformed_ratios = ['quick_ratio_reciprocal', 'debt_to_equity_ratio_log', 'long_term_debt_to_assets_reciprocal', 
                      'working_capital_to_assets_sqrt', 'z_score_log', 'market_to_book_log', 'return_on_equity_log']
print(df[transformed_ratios].head())

# Check for any remaining infinite values
print("\nChecking for infinite values in transformed ratios:")
for col in transformed_ratios:
    inf_count = np.isinf(df[col]).sum()
    if inf_count > 0:
        print(f"{col} has {inf_count} infinite values")
    else:
        print(f"{col} has no infinite values")


Transformed financial ratios - first few rows:
   quick_ratio_reciprocal  debt_to_equity_ratio_log  \
0                0.483140                  0.780480   
1                0.431407                  0.724156   
2                0.500659                  0.828215   
3                0.598692                  0.844848   
4                0.415094                  0.854887   

   long_term_debt_to_assets_reciprocal  working_capital_to_assets_sqrt  \
0                             0.804170                       79.520305   
1                             0.795896                       79.520586   
2                             0.765385                       79.519890   
3                             0.806576                       79.519123   
4                             0.740421                       79.520024   

   z_score_log  market_to_book_log  return_on_equity_log  
0    10.582482           10.519441             10.368419  
1    10.582480           10.519441             10.368418  

Handling Infinite and Missing Values

Financial ratios can sometimes produce infinite or missing values due to division by zero or other issues. We'll handle these values to ensure data quality.


In [13]:
# Replace inf and -inf with NaN
df.replace([float('inf'), -float('inf')], np.nan, inplace=True)

# List all columns that need imputation
columns_to_impute = [
    'current_ratio', 'quick_ratio', 'debt_to_equity_ratio', 'long_term_debt_to_assets',
    'profit_margin', 'gross_margin', 'return_on_assets', 'return_on_equity',
    'asset_turnover', 'working_capital_to_assets', 'retained_earnings_to_assets',
    'ebit_to_assets', 'market_value_to_liabilities', 'sales_to_assets', 'z_score',
    'market_to_book',
    'quick_ratio_reciprocal', 'debt_to_equity_ratio_log', 'long_term_debt_to_assets_reciprocal',
    'working_capital_to_assets_sqrt', 'z_score_log', 'market_to_book_log', 'return_on_equity_log'
]

# Check for missing values before imputation
print("\nMissing values before imputation:")
print(df[columns_to_impute].isna().sum())

# Impute missing values with median for each column
for column in columns_to_impute:
    if df[column].isna().sum() > 0:
        df[column] = df[column].fillna(df[column].median())

# Verify imputation
print("\nMissing values after imputation:")
print(df[columns_to_impute].isna().sum())


Missing values before imputation:
current_ratio                          0
quick_ratio                            0
debt_to_equity_ratio                   2
long_term_debt_to_assets               0
profit_margin                          0
gross_margin                           0
return_on_assets                       0
return_on_equity                       2
asset_turnover                         0
working_capital_to_assets              0
retained_earnings_to_assets            0
ebit_to_assets                         0
market_value_to_liabilities            0
sales_to_assets                        0
z_score                                0
market_to_book                         2
quick_ratio_reciprocal                 0
debt_to_equity_ratio_log               2
long_term_debt_to_assets_reciprocal    0
working_capital_to_assets_sqrt         0
z_score_log                            0
market_to_book_log                     2
return_on_equity_log                   0
dtype: int64

Missing 

Creating Event and Duration Variables for Survival Analysis

For survival analysis models like Cox Proportional Hazards, we need event and duration variables.


In [14]:
# Create 'event' variable from target_last
df['event'] = df['target_last']

# Create 'duration' variable (time from first to last year for each company)
df['duration'] = df.groupby('company_name')['year'].transform(lambda x: x.max() - x.min() + 1)

# Display the event and duration variables
print("\nEvent and duration variables - first few rows:")
print(df[['company_name', 'year', 'event', 'duration']].head())



Event and duration variables - first few rows:
  company_name  year  event  duration
0          C_1  1999      0        19
1          C_1  2000      0        19
2          C_1  2001      0        19
3          C_1  2002      0        19
4          C_1  2003      0        19


Final Dataset Review

In [15]:
# List of all columns in the final dataset
final_columns = [
    'company_name', 'status_label', 'year', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10',
    'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'target_last', 'X8_log', 'X8_boxcox',
    'X10_log', 'X10_boxcox', 'left_censored', 'period_after_2008', 'current_ratio', 'quick_ratio',
    'debt_to_equity_ratio', 'long_term_debt_to_assets', 'profit_margin', 'gross_margin',
    'return_on_assets', 'return_on_equity', 'asset_turnover', 'working_capital_to_assets',
    'retained_earnings_to_assets', 'ebit_to_assets', 'market_value_to_liabilities', 'sales_to_assets',
    'z_score', 'bankruptcy', 'market_to_book', 'quick_ratio_reciprocal', 'debt_to_equity_ratio_log',
    'long_term_debt_to_assets_reciprocal', 'working_capital_to_assets_sqrt', 'z_score_log',
    'market_to_book_log', 'return_on_equity_log', 'return_on_assets_shifted', 'profit_margin_shifted',
    'event', 'duration'
]

In [16]:
# Ensure all expected columns exist in the dataframe
missing_columns = [col for col in final_columns if col not in df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
    # Create missing columns with placeholder values
    for col in missing_columns:
        df[col] = np.nan

In [17]:
# Final dataset structure
print("\nFinal dataset structure:")
print(df[final_columns].head())


Final dataset structure:
  company_name status_label  year       X1       X2      X3      X4       X5  \
0          C_1        alive  1999  511.267  833.107  18.373  89.031  336.018   
1          C_1        alive  2000  485.856  713.811  18.577  64.367  320.590   
2          C_1        alive  2001  436.656  526.477  22.496  27.207  286.588   
3          C_1        alive  2002  396.412  496.747  27.172  30.745  259.954   
4          C_1        alive  2003  432.204  523.302  26.680  47.491  247.245   

       X6       X7  ...  debt_to_equity_ratio_log  \
0  35.163  128.348  ...                  0.780480   
1  18.531  115.187  ...                  0.724156   
2 -58.939   77.528  ...                  0.828215   
3 -12.410   66.322  ...                  0.844848   
4   3.504  104.661  ...                  0.854887   

   long_term_debt_to_assets_reciprocal  working_capital_to_assets_sqrt  \
0                             0.804170                       79.520305   
1                         

In [18]:
# Verify column count matches expected count
print(f"\nNumber of columns in final dataset: {len(df.columns)}")
print(f"Number of expected columns: {len(final_columns)}")




Number of columns in final dataset: 56
Number of expected columns: 56


In [19]:
# Save the final transformed dataset
df[final_columns].to_csv('american_bankruptcy_transformed.csv', index=False)
print("\nTransformed dataset saved to 'american_bankruptcy_transformed.csv'")



Transformed dataset saved to 'american_bankruptcy_transformed.csv'
